In [2]:
# ORDER Types
# buy
# sell
# stoploss
# cancel

# no short selling
# 1 hour real time data = 1 second in simulation
# find data source?

# test diff to real stocks?
# Real day has different stocks and data to the testing, so could potentially find it but they are anonymised...
# Scale the testing data by 3600 then match it to known hitorical data
# Data given in pandas dataframe
# all trades made at bid (for selling our stock) and ask (for buying) prices
# no fee to trade
# infinite stocks, we arent trading with other competitors. Everything is pre calculated.
# Account can go to zero, finite money. Certain judges have good will which they can give money to you if feeling kind.
# Can buy unlimited volume, so good strategy is to buy extremely large amount of safe stock 

# Import notes:
# the live data as per the API has accuracy to two DP, but the historic csv has much more. ---> careful when training


# IDEAS
# - during the event we can train models off the live data and update incrementally
# want everything running concurrently to improve speed

In [1]:
import hackathon_linc as lh
import pandas as pd
import time
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
lh.init('265d0b0b-7e97-44a7-9576-47789e8712b2')



    Welcome to the LINC Hackathon! Your token is now saved in the Console. 
    That means you don't need to carry that out when using the other functions
    as long as you don't close your console. 
    
    This function is only to be used once to authenticate your token.
    
    Happy coding!

    


In [7]:
start_time = time.time()
orders = lh.get_all_orders()
print(f"Orders: {orders} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
completed = lh.get_completed_orders()
print(f"Completed Orders: {completed} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
pending = lh.get_pending_orders()
print(f"Pending Orders: {pending} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
stoploss_orders = lh.get_stoploss_orders()
print(f"Stoploss Orders: {stoploss_orders} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
balance = lh.get_balance()
print(f"Balance: {balance} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
portfolio = lh.get_portfolio()
print(f"Portfolio: {portfolio} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
all_tickers = lh.get_all_tickers()
print(f"All Tickers: {all_tickers} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
price_data = pd.DataFrame(lh.get_current_price()['data'])
display(price_data)
print(f"Time taken: {time.time() - start_time:.2f} seconds")


Orders: [{'amount': 1, 'created_at': '2015-12-18T07:51:00', 'ended_at': '2015-12-18T07:51:00', 'id': 'fed18efa-7a3b-4bce-84bd-8ec4336d250a', 'order_status': 'completed', 'order_type': 'buy', 'price': 91.247, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062ef4f', 'symbol': 'STOCK1', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}, {'amount': 3, 'created_at': '2016-04-14T11:30:00', 'ended_at': '2016-04-14T11:30:00', 'id': '2d3a5c26-6086-4704-b48b-c56060ae1df8', 'order_status': 'completed', 'order_type': 'buy', 'price': 78.28244166666667, 'security_id': 'dc708dd9-f3a0-4a00-b871-43e58d4948e1', 'symbol': 'STOCK3', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}, {'amount': 1, 'created_at': '2015-12-18T09:05:00', 'ended_at': '2015-12-18T09:05:00', 'id': '839bfdaf-d373-4252-97a5-c9a56443a5ca', 'order_status': 'completed', 'order_type': 'buy', 'price': 90.16170833333332, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062ef4f', 'symbol': 'STOCK1', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c

,askMedian,askVolume,bidMedian,bidVolume,gmtTime,spreadMedian,symbol
0,115.46,6.658624e+04,115.39,8.403509e+04,2016-05-27 08:00:00,0.068189,STOCK1
1,134.04,6.902538e+04,133.96,4.834713e+04,2016-05-27 08:00:00,0.077008,STOCK10
2,69.93,9.889253e+04,69.91,9.422933e+04,2016-05-27 08:00:00,0.024450,STOCK2
3,75.08,1.098172e+06,75.05,1.114719e+06,2016-05-27 08:00:00,0.024758,STOCK3
4,55.03,4.096056e-02,55.00,3.100464e+04,2016-05-27 08:00:00,0.032462,STOCK4
5,16.12,1.134240e+06,16.10,1.099421e+06,2016-05-27 08:00:00,0.017542,STOCK5
6,104.45,6.154389e+04,104.38,5.670475e+04,2016-05-27 08:00:00,0.067625,STOCK6
7,13.26,1.106887e+06,13.25,1.282287e+06,2016-05-27 08:00:00,0.017058,STOCK7
8,72.73,7.875523e-02,72.70,7.945962e+04,2016-05-27 08:00:00,0.022845,STOCK8
9,97.79,5.265883e+04,97.77,6.641300e+04,2016-05-27 08:00:00,0.024142,STOCK9


Time taken: 0.29 seconds


In [ ]:
def buy_stock():
    return lh.buy("STOCK1", 1, None, None)

def sell_stock():
    return lh.sell("STOCK1", 1, None, None)

max_workers = 1

for i in range(20):
    print(f"~~~~~~~~~~~~~~~~{i}~~~~~~~~~~~~~~~~")
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=max_workers*2) as executor:
        futures = [executor.submit(buy_stock) for _ in range(max_workers)] + [executor.submit(sell_stock) for _ in range(max_workers)]
        for future in as_completed(futures):
            print(future.result())
    elapsed_time = time.time() - start_time
    if elapsed_time < 1:
        time.sleep(1 - elapsed_time)
    
    

    
    

~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~
{'amount': 1, 'created_at': '2015-12-23T15:25:00', 'ended_at': '2015-12-23T15:25:00', 'id': '0cccafd4-e356-4b89-a545-f8cd247c62f6', 'order_status': 'completed', 'order_type': 'buy', 'price': 90.13358333333332, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062ef4f', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}
{'amount': 1, 'created_at': '2015-12-23T15:31:00', 'ended_at': '2015-12-23T15:32:00', 'id': 'c2808238-82b6-4aab-a1d7-ba2b104858de', 'order_status': 'completed', 'order_type': 'sell', 'price': 90.13358333333332, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062ef4f', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}
~~~~~~~~~~~~~~~~1~~~~~~~~~~~~~~~~
{'message': 'Something else went wrong'}
{'amount': 1, 'created_at': '2015-12-24T07:13:00', 'ended_at': '2015-12-24T07:14:00', 'id': 'cc288297-dc70-412a-80c1-4185f9eb6b2f', 'order_status': 'completed', 'order_type': 'buy', 'price': 90.17190322580645, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062

In [4]:
lh.buy("STOCK3", 3, None, None)

{'amount': 3,
 'created_at': '2016-04-14T11:30:00',
 'ended_at': '2016-04-14T11:30:00',
 'id': '2d3a5c26-6086-4704-b48b-c56060ae1df8',
 'order_status': 'completed',
 'order_type': 'buy',
 'price': 78.28244166666667,
 'security_id': 'dc708dd9-f3a0-4a00-b871-43e58d4948e1',
 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}

In [6]:
lh.sell("STOCK3", 3, None, None)

{'amount': 3,
 'created_at': '2016-04-18T09:03:00',
 'ended_at': '2016-04-18T09:03:00',
 'id': '9a0d9535-8752-44d9-b725-4c7baa3dff0a',
 'order_status': 'completed',
 'order_type': 'sell',
 'price': 79.2978,
 'security_id': 'dc708dd9-f3a0-4a00-b871-43e58d4948e1',
 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}

In [3]:
portfolio = lh.get_portfolio()
portfolio

{'STOCK1': 60}

In [6]:
start_time = time.time()
all_tickers = lh.get_all_tickers()
print(f"All Tickers: {all_tickers} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
price_data = pd.DataFrame(lh.get_current_price()['data'])
display(price_data)
print(f"Time taken: {time.time() - start_time:.2f} seconds")



All Tickers: ['STOCK1', 'STOCK10', 'STOCK2', 'STOCK3', 'STOCK4', 'STOCK5', 'STOCK6', 'STOCK7', 'STOCK8', 'STOCK9'] (Time taken: 0.15 seconds)


,askMedian,askVolume,bidMedian,bidVolume,gmtTime,spreadMedian,symbol
0,66.85,219.033333,66.81,181.650000,2015-09-08 10:00:00,0.032108,STOCK1
1,167.74,102922.833820,167.66,106690.082447,2015-09-08 10:00:00,0.077567,STOCK10
2,71.57,153353.983180,71.55,150883.150563,2015-09-08 10:00:00,0.022317,STOCK2
3,84.17,121623.633393,84.15,146294.666270,2015-09-08 10:00:00,0.027100,STOCK3
4,101.65,2321.450012,101.54,3216.766638,2015-09-08 10:00:00,0.110792,STOCK4
5,26.52,428378.915668,26.51,460617.850752,2015-09-08 10:00:00,0.010183,STOCK5
6,93.34,18423.699908,93.30,24983.966722,2015-09-08 10:00:00,0.039508,STOCK6
7,10.01,274489.633088,10.00,197197.949932,2015-09-08 10:00:00,0.006275,STOCK7
8,59.30,90438.817005,59.28,131240.733652,2015-09-08 10:00:00,0.019733,STOCK8
9,87.75,105250.449617,87.72,99506.533885,2015-09-08 10:00:00,0.023683,STOCK9


Time taken: 0.15 seconds


In [5]:
historical_data = pd.DataFrame(lh.get_historical_data(100))
print(historical_data)

       askMedian     askVolume   bidMedian     bidVolume              gmtTime  \
0      21.717000  1.000000e+03   21.703000  1.000000e+03  2015-12-21 07:00:00   
1      89.070000  1.000000e+03   89.060000  1.000000e+03  2015-12-21 07:00:00   
2      96.694000  1.000000e+03   96.642000  1.000000e+03  2015-12-21 07:00:00   
3     129.638000  1.000000e+03  129.553000  1.000000e+03  2015-12-21 07:00:00   
4      89.444000  1.000000e+03   89.396000  1.000000e+03  2015-12-21 07:00:00   
...          ...           ...         ...           ...                  ...   
6432   96.996917  2.370000e+01   96.970317  2.370000e+01  2016-03-29 13:00:00   
6433   70.323583  3.978333e+01   70.309167  3.978333e+01  2016-03-29 13:00:00   
6434   65.812517  4.981667e+01   65.798500  4.981667e+01  2016-03-29 13:00:00   
6435  102.307117  1.037084e+06  102.255250  1.096772e+06  2016-03-29 13:00:00   
6436   15.246917  1.613333e+01   15.238550  1.613333e+01  2016-03-29 13:00:00   

      spreadMedian   symbol

In [4]:
lh.get_historical_data(25)

[{'askMedian': 11.05675,
  'askVolume': 2917115.913626923,
  'bidMedian': 11.04875,
  'bidVolume': 2855385.168880769,
  'gmtTime': '2016-02-25 11:00:00',
  'spreadMedian': 0.0080288461538461,
  'symbol': 'STOCK7'},
 {'askMedian': 83.74833333333332,
  'askVolume': 2237651.1275833333,
  'bidMedian': 83.7321,
  'bidVolume': 2165243.224301667,
  'gmtTime': '2016-02-25 11:00:00',
  'spreadMedian': 0.0162583333333333,
  'symbol': 'STOCK9'},
 {'askMedian': 70.94761666666666,
  'askVolume': 2239207.1107983333,
  'bidMedian': 70.9293,
  'bidVolume': 2404024.492953333,
  'gmtTime': '2016-02-25 11:00:00',
  'spreadMedian': 0.0183583333333333,
  'symbol': 'STOCK3'},
 {'askMedian': 99.28558333333334,
  'askVolume': 693361.5016466667,
  'bidMedian': 99.24298333333331,
  'bidVolume': 788995.7981833334,
  'gmtTime': '2016-02-25 11:00:00',
  'spreadMedian': 0.0425583333333333,
  'symbol': 'STOCK10'},
 {'askMedian': 58.75736666666667,
  'askVolume': 2593401.3754166667,
  'bidMedian': 58.7406,
  'bidVolu

### Overall controller class

In [13]:
class DataPoller:
    def __init__(self, polling_intervals):
        self.polling_intervals = polling_intervals
        self.cache = {}
        self.running = False
        self.executor = ThreadPoolExecutor(max_workers=len(polling_intervals))

    def poll_data(self, func, func_name, interval):
        while self.running:
            start_time = time.time()
            result = func()
            self.cache[func_name] = result
            elapsed_time = time.time() - start_time
            print(f"{func_name}: {result} (Time taken: {elapsed_time:.2f} seconds)")
            time.sleep(interval)

    def start_polling(self):
        self.running = True
        for func_name, interval in self.polling_intervals.items():
            func = getattr(lh, func_name)
            self.executor.submit(self.poll_data, func, func_name, interval)

    def stop_polling(self):
        self.running = False
        self.executor.shutdown(wait=False)

    def get_cached_data(self, func_name):
        return self.cache.get(func_name)

def strategy_function(data_poller):
    while True:
        # Access cached data
        all_orders = data_poller.get_cached_data("get_all_orders")
        completed_orders = data_poller.get_cached_data("get_completed_orders")
        pending_orders = data_poller.get_cached_data("get_pending_orders")
        stoploss_orders = data_poller.get_cached_data("get_stoploss_orders")
        balance = data_poller.get_cached_data("get_balance")
        portfolio = data_poller.get_cached_data("get_portfolio")
        all_tickers = data_poller.get_cached_data("get_all_tickers")
        current_price = data_poller.get_cached_data("get_current_price")

        # Implement your strategy logic here
        print(f"Strategy accessing data: {all_orders}, {completed_orders}, {pending_orders}, {stoploss_orders}, {balance}, {portfolio}, {all_tickers}, {current_price}")

        time.sleep(1)

# Example usage
if __name__ == "__main__":
    polling_intervals = {
        "get_all_orders": 5,
        "get_completed_orders": 10,
        "get_pending_orders": 15,
        "get_stoploss_orders": 5,
        "get_balance": 10,
        "get_portfolio": 15,
        "get_all_tickers": 2,
        "get_current_price": 1,
    }

    data_poller = DataPoller(polling_intervals)

    try:
        data_poller.start_polling()
        while True:
            time.sleep(1)
            print(1)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(2)
            print(2)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(3)
            print(3)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(4)
            print(4)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(5)
            print(5)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(6)
            print(6)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
    except KeyboardInterrupt:
        data_poller.stop_polling()

get_completed_orders: [] (Time taken: 0.24 seconds)
get_pending_orders: [] (Time taken: 0.29 seconds)
get_portfolio: {} (Time taken: 0.29 seconds)
get_all_tickers: ['STOCK1', 'STOCK10', 'STOCK2', 'STOCK3', 'STOCK4', 'STOCK5', 'STOCK6', 'STOCK7', 'STOCK8', 'STOCK9'] (Time taken: 0.30 seconds)
get_stoploss_orders: [] (Time taken: 0.31 seconds)
get_current_price: {'data': [{'askMedian': 87.54, 'askVolume': 273.35, 'bidMedian': 87.51, 'bidVolume': 297.5833333333333, 'gmtTime': '2015-12-10 09:00:00', 'spreadMedian': 0.036725, 'symbol': 'STOCK1'}, {'askMedian': 135.8, 'askVolume': 121964.25022333332, 'bidMedian': 135.69, 'bidVolume': 70119.33303166667, 'gmtTime': '2015-12-10 09:00:00', 'spreadMedian': 0.110275, 'symbol': 'STOCK10'}, {'askMedian': 70.75, 'askVolume': 100278.11620666667, 'bidMedian': 70.73, 'bidVolume': 94534.33401833332, 'gmtTime': '2015-12-10 09:00:00', 'spreadMedian': 0.02165, 'symbol': 'STOCK2'}, {'askMedian': 97.39, 'askVolume': 44902.78333666667, 'bidMedian': 97.33, 'bid

get_pending_orders: [] (Time taken: 0.16 seconds)
get_portfolio: {} (Time taken: 0.12 seconds)
get_completed_orders: [] (Time taken: 0.20 seconds)
get_balance: 1000000.0 (Time taken: 0.14 seconds)
get_all_orders: [] (Time taken: 0.32 seconds)
get_stoploss_orders: [] (Time taken: 0.17 seconds)
get_all_orders: [] (Time taken: 0.29 seconds)
get_stoploss_orders: [] (Time taken: 0.15 seconds)
get_completed_orders: [] (Time taken: 0.19 seconds)
get_balance: 1000000.0 (Time taken: 0.14 seconds)
get_all_orders: [] (Time taken: 0.36 seconds)
get_stoploss_orders: [] (Time taken: 0.14 seconds)
get_pending_orders: [] (Time taken: 0.13 seconds)
get_portfolio: {} (Time taken: 0.12 seconds)
get_all_orders: [] (Time taken: 0.25 seconds)
get_stoploss_orders: [] (Time taken: 0.13 seconds)
get_completed_orders: [] (Time taken: 0.16 seconds)
get_balance: 1000000.0 (Time taken: 0.22 seconds)
get_stoploss_orders: [] (Time taken: 0.13 seconds)
get_stoploss_orders: [] (Time taken: 0.13 seconds)
get_pending_or

In [17]:
data_poller.stop_polling()

get_completed_orders: [] (Time taken: 0.14 seconds)


get_balance: 1000000.0 (Time taken: 0.13 seconds)
get_stoploss_orders: [] (Time taken: 0.21 seconds)
get_pending_orders: [] (Time taken: 0.14 seconds)
get_portfolio: {} (Time taken: 0.13 seconds)
get_stoploss_orders: [] (Time taken: 0.11 seconds)
get_completed_orders: [] (Time taken: 0.21 seconds)
get_balance: 1000000.0 (Time taken: 0.15 seconds)
get_stoploss_orders: [] (Time taken: 0.16 seconds)
